In [1]:
'''
Make scorecard
'''
import os
from datetime import datetime, date
import pandas as pd
import numpy as np
# import openpyxl
# from openpyxl import load_workbook
from dateutil.relativedelta import relativedelta

In [4]:
#Set file locations
print('Grabbing file names...')
PPD_FILE = '../../../Data/PPD/ppd_data_20200404.csv'
WSLIVE_FILE = '../../../Data/Scorecard/p-WSLive-Results-20200311.xlsm'
OLD_SCORECARD = '../../../Data/Scorecard/v-Masterfile-Scorecard-February_2020-03-05.xlsx'
EMAIL_FILE = '../../../Data/Scorecard/Email Availability Monthly Count.xlsx'
DO_FILE = '../../../Data/Scorecard/do_count.txt'

Grabbing file names...


In [44]:
#Set today


def get_email_data(email_file):
    '''Get email data'''
    em_tbl = pd.read_excel(email_file, header=2)
    email_count = int(em_tbl.iloc[2,1])
    email_correctness = em_tbl.iloc[3,1]
    return(email_count, email_correctness)

def get_do_counts(do_file):
    '''Get do counts'''
    with open(do_file, 'r') as file_:
        read_file = file_.read()
        phys_count = int(read_file.split('\n')[0].replace('Physicians: ', '').replace(',', ''))
        stu_count = int(read_file.split('\n')[1].replace('Students: ', '').replace(',', ''))
    return phys_count, stu_count

def read_ppd(ppd_file_location):
    '''Read ppd'''
    ppd = pd.read_csv(ppd_file_location, low_memory=False)

    #Nones
    ppd['ADDRESS_UNDELIVERABLE_FLAG_2'] = ppd['ADDRESS_UNDELIVERABLE_FLAG'].fillna('None')
    ppd['PRESUMED_DEAD_FLAG_2'] = ppd['PRESUMED_DEAD_FLAG'].fillna('None')

    #Everybody gotta be alive
    ppd = ppd[ppd['PRESUMED_DEAD_FLAG_2'] == 'None']

    #Define subset of DOs
    sub_ppd = ppd[ppd.TOP_CD == 20]

    return(ppd, sub_ppd)

#Total
def get_total_table(ppd_df):
    '''Get complete counts'''
    all_deliverable = ppd_df[ppd_df['ADDRESS_UNDELIVERABLE_FLAG_2'] == 'None']
    counts = ppd_df.count()
    fax_count = counts['FAX_NUMBER']
    polo_count = counts['POLO_MAILING_LINE_2']
    telephone_count = counts['TELEPHONE_NUMBER']
    # no_delivery_count = counts['ADDRESS_UNDELIVERABLE_FLAG']
    deliverable_counts = all_deliverable.count()
    mailing_address_count = deliverable_counts['MAILING_LINE_2']
    top_count = len(ppd_df[ppd_df['TOP_CD'] != 100])
    pe_cd_count = len(ppd_df[ppd_df['PE_CD'] != 110])
    prim_spec_count = len(ppd_df[ppd_df['PRIM_SPEC_CD'] != 'US'])
    total_records = len(ppd_df)

    value_dict = {
        'totalrecords':total_records,
        'mailingaddress':mailing_address_count,
        'polo':polo_count,
        'telephone':telephone_count,
        'faxnumber':fax_count,
        'top':top_count,
        'pe':pe_cd_count,
        'primspec':prim_spec_count
    }

    transposed = pd.DataFrame(value_dict, ['Count']).transpose()

    return transposed

def get_wslive_results(wslive_file_location):
    '''Read WSLive Results'''
    wslive_table = pd.read_excel(wslive_file_location, sheet_name='Summary Percentage', header=3).dropna()
    values = list(wslive_table[wslive_table['POLO Address Status'] == 'Confirmed'].iloc[:, -1])
    values.remove(1)

    col_list = ['polo_correct',
                'employment_correct',
                'telephone_correct',
                'telephone_2_correct',
                'fax_correct']

    wslive_df = pd.DataFrame({'1':col_list, '2':values})

    return wslive_df

def read_old_scorecard(old_scorecard_location):
    '''Read old scorecard'''
    notes = pd.read_excel(old_scorecard_location, sheet_name='Notes')
    notes_2 = pd.read_excel(old_scorecard_location, sheet_name='Notes_2')
    notes_3 = pd.read_excel(old_scorecard_location, sheet_name='Notes_3')

    return(notes, notes_2, notes_3)

def get_notes(total, sub, correct, email, physicians, students):
    '''Get notes'''
    totalrecords = int(total.loc['totalrecords'])
    mailingaddress = int(total.loc['mailingaddress'])
    polo = int(total.loc['polo'])
    telephone = int(total.loc['telephone'])
    faxnumber = int(total.loc['faxnumber'])
    top = int(total.loc['top'])
    pe = int(total.loc['pe'])
    primspec = int(total.loc['primspec'])

    totalrecords_dpc = int(sub.loc['totalrecords'])
    mailingaddress_dpc = int(sub.loc['mailingaddress'])
    polo_dpc = int(sub.loc['polo'])
    telephone_dpc = int(sub.loc['telephone'])
    faxnumber_dpc = int(sub.loc['faxnumber'])
    top_dpc = int(sub.loc['top'])
    pe_dpc = int(sub.loc['pe'])
    primspec_dpc = int(sub.loc['primspec'])

    polo_correctness = float(correct.iloc[0, 1])
    phone_correctness = float(correct.iloc[2, 1])
    phone_2_correctness = float(correct.iloc[3, 1])

    email_count = get_email_data(email)[0]
    email_correctness = get_email_data(email)[1]

    mailing_completeness = mailingaddress/totalrecords
    polo_completeness = polo_dpc/totalrecords_dpc
    phone_completeness = telephone_dpc/totalrecords_dpc
    fax_completeness = faxnumber_dpc/totalrecords_dpc
    practice_compelteness = top/totalrecords
    employment_completeness = pe/totalrecords
    spec_completeness = primspec/totalrecords
    email_completeness = email_count/totalrecords

    polo_combo = polo_correctness * polo_completeness
    phone_combo = phone_correctness * phone_completeness
    email_combo = email_correctness * email_completeness

    polo_correct = int(polo_combo * totalrecords_dpc)
    phone_correct = int(phone_combo * totalrecords_dpc)
    email_corect = int(email_combo * totalrecords)

    do_student = students
    do_physician = physicians
    do_student_total = int(2.898100e+04)
    do_physician_total = int(1.081180e+05)
    all_ = f'All Physicians: {"{:,}".format(totalrecords)} Records'
    all_dpc = f'Direct Patient Care (DPC): {"{:,}".format(totalrecords_dpc)} Records'

    notes_list_ = [totalrecords,
                   mailingaddress,
                   polo,
                   telephone,
                   faxnumber,
                   top,
                   pe,
                   primspec,
                   email_count,
                   totalrecords_dpc,
                   mailingaddress_dpc,
                   polo_dpc,
                   telephone_dpc,
                   faxnumber_dpc,
                   top_dpc,
                   pe_dpc,
                   primspec_dpc,
                   polo_correctness,
                   phone_correctness,
                   phone_2_correctness,
                   email_correctness,
                   do_student,
                   do_physician,
                   do_student_total,
                   do_physician_total,
                   all_,
                   all_dpc
                   ]
    notes_2_list_ = [mailing_completeness,
                     polo_completeness,
                     phone_completeness,
                     fax_completeness,
                     practice_compelteness,
                     employment_completeness,
                     spec_completeness,
                     email_completeness,
                     polo_correctness,
                     phone_correctness,
                     phone_2_correctness,
                     email_correctness,
                     polo_correct,
                     phone_correct,
                     email_corect,
                     polo_combo,
                     phone_combo,
                     email_combo,
                     do_student,
                     do_physician,
                     totalrecords,
                     totalrecords_dpc]
    return (notes_list_, notes_2_list_)

def get_dates():
    '''Get relevant dates'''
    today_date = datetime.today()
    scorecard_month = (today_date - relativedelta(months=1)).strftime("%b" " " "%Y")
    last_month = (today_date - relativedelta(months=2)).strftime("%b" " " "%Y")
    last_year = (today_date - relativedelta(months=1, years=1)).strftime("%b" " " "%Y")
    two_year = (today_date - relativedelta(months=1, years=2)).strftime("%b" " " "%Y")
    return(scorecard_month, last_month, last_year, two_year)

def get_new_notes(notes_list, notes_2_list, notes, notes_2, notes_3):
    '''Get new notes'''
    scorecard_month, last_month, last_year, two_year = get_dates()
    notes_list.append(f'Cumulative Change from {last_year}')
    notes_list.append(f'Cumulative Change from {two_year}')
    notes.COL1 = notes_list
    notes_3[last_month] = list(notes_2[last_month])
#     notes_3 = notes_3.drop(columns=last_month)

#     notes_2 = notes_2.drop(columns=last_year)
    notes_2[scorecard_month] = notes_2_list
    standard_devs = []
    for num in list(range(0, 22)):
        standard_devs.append(np.std(notes_2.iloc[[num], 2:15].values, dtype=np.float64))
    notes_2.Standard_Deviation = standard_devs
    return(notes, notes_2, notes_3)

# print('Reading ppd...')
# PPD, SUB_PPD = read_ppd(PPD_FILE)
# print('Getting ppd data...')
# TOTAL = get_total_table(PPD)
# SUB = get_total_table(SUB_PPD)

In [60]:
print('Getting wslive data...')
CORRECT = get_wslive_results(WSLIVE_FILE)
print('Reading old scorecard...')
NOTES, NOTES_2, NOTES_3 = read_old_scorecard(OLD_SCORECARD)
print('Getting DO counts...')
PHYSICIANS, STUDENTS = get_do_counts(DO_FILE)
print('Getting new notes...')

Getting wslive data...
Reading old scorecard...
Getting DO counts...
Getting new notes...


In [61]:
NOTE_LIST, NOTE_LIST_2 = get_notes(TOTAL, SUB, CORRECT, EMAIL_FILE, PHYSICIANS, STUDENTS)
print('Editing notes...')
NEW_NOTES, NEW_NOTES_2, NEW_NOTES_3 = get_new_notes(NOTE_LIST, NOTE_LIST_2, NOTES, NOTES_2, NOTES_3)
print('Writing to file...')
# with pd.ExcelWriter(f'{LOCAL_DIRECTORY}output.xlsx') as writer:  
#     NEW_NOTES.to_excel(writer, sheet_name='Notes', index=False)
#     NEW_NOTES_2.to_excel(writer, sheet_name='Notes_2', index=False)
#     NEW_NOTES_3.to_excel(writer, sheet_name='Notes_3', index=False)

Editing notes...
Writing to file...


In [23]:
NOTE_LIST

[1257135,
 1232980,
 985321,
 773661,
 642837,
 1173217,
 957677,
 1210166,
 860992,
 815406,
 809320,
 727437,
 668284,
 558659,
 815406,
 646884,
 799759,
 0.7835671342685371,
 0.4590909090909091,
 0.5937295885042456,
 0.82558374526128,
 8240,
 91845,
 28981,
 108118,
 'All Physicians: 1,257,135 Records',
 'Direct Patient Care (DPC): 815,406 Records',
 'Cumulative Change from Feb 2019',
 'Cumulative Change from Feb 2018']

In [24]:
NOTES

,_NAME_,COL1
0,totalrecords,1257135
1,mailingaddress,1232980
2,polo,985321
3,telephone,773661
4,faxnumber,642837
5,top,1173217
6,pe,957677
7,primspec,1210166
8,email,860992
9,totalrecords_dpc,815406


In [48]:
NEW_NOTES_2

,Core Existing Data,Standard_Deviation,Feb 2019,Apr 2019,May 2019,Jun 2019,Jul 2019,Aug 2019,Sep 2019,Oct 2019,Nov 2019,Dec 2019,Jan 2020,Feb 2020,Mar 2020
0,mailing_completeness,0.000934,9.801475e-01,9.793197e-01,9.820495e-01,9.823056e-01,9.824251e-01,9.824004e-01,9.823522e-01,9.821872e-01,9.820343e-01,9.820186e-01,9.816848e-01,9.813482e-01,9.807857e-01
1,polo_completeness,0.006897,9.127862e-01,9.093118e-01,9.097503e-01,9.090362e-01,9.074398e-01,9.065507e-01,9.032298e-01,9.014950e-01,8.986148e-01,8.957083e-01,8.938377e-01,8.933039e-01,8.921163e-01
2,phone_completeness,0.024785,7.695343e-01,7.748903e-01,7.662985e-01,7.731660e-01,7.723157e-01,7.847550e-01,7.758147e-01,7.936667e-01,8.230647e-01,8.385015e-01,8.178283e-01,8.239510e-01,8.195721e-01
3,fax_completeness,0.008432,6.590192e-01,6.647006e-01,6.677837e-01,6.712995e-01,6.750114e-01,6.835850e-01,6.804054e-01,6.808750e-01,6.823712e-01,6.844692e-01,6.842981e-01,6.843971e-01,6.851299e-01
4,practice_compelteness,0.003032,9.368782e-01,9.370184e-01,9.382197e-01,9.385640e-01,9.294054e-01,9.286639e-01,9.310614e-01,9.323248e-01,9.327903e-01,9.331049e-01,9.334729e-01,9.332388e-01,9.332466e-01
5,employment_completeness,0.005332,7.610338e-01,7.656256e-01,7.707130e-01,7.718279e-01,7.615926e-01,7.560883e-01,7.524973e-01,7.554930e-01,7.579662e-01,7.601435e-01,7.609208e-01,7.610465e-01,7.617933e-01
6,spec_completeness,0.007359,9.646244e-01,9.635262e-01,9.477665e-01,9.475002e-01,9.477912e-01,9.489178e-01,9.496063e-01,9.629498e-01,9.634377e-01,9.634724e-01,9.633218e-01,9.630815e-01,9.626381e-01
7,email_completeness,0.020126,6.401563e-01,6.424602e-01,6.333600e-01,6.347998e-01,6.365401e-01,6.409586e-01,6.444066e-01,6.658347e-01,6.675823e-01,6.810460e-01,6.824598e-01,6.832901e-01,6.848843e-01
8,polo_correctness,0.031917,8.322926e-01,8.210432e-01,8.370000e-01,8.541667e-01,8.641860e-01,8.428710e-01,8.435973e-01,8.415842e-01,8.392051e-01,9.162349e-01,8.586387e-01,7.884085e-01,7.835671e-01
9,phone_correctness,0.022886,5.299446e-01,5.300050e-01,5.191000e-01,5.362615e-01,5.145186e-01,4.929078e-01,4.928789e-01,5.133929e-01,5.200793e-01,4.735529e-01,5.345000e-01,5.091278e-01,4.590909e-01


In [49]:
TODAY = date.today()
scorecard_month = (TODAY - relativedelta(months=1)).strftime("%b" " " "%Y")
last_month = (TODAY - relativedelta(months=2)).strftime("%b" " " "%Y")
last_year = (TODAY - relativedelta(months=2, years=1)).strftime("%b" " " "%Y")
two_year = (TODAY - relativedelta(months=2, years=2)).strftime("%b" " " "%Y")

In [35]:
two_year

'Feb 2018'

In [51]:
NEW_NOTES

,_NAME_,COL1
0,totalrecords,1257135
1,mailingaddress,1232980
2,polo,985321
3,telephone,773661
4,faxnumber,642837
5,top,1173217
6,pe,957677
7,primspec,1210166
8,email,860992
9,totalrecords_dpc,815406


In [55]:
NEW_NOTES_3.columns

Index(['Unnamed: 0', 'Feb 2018', 'Mar 2018', 'Apr 2018', 'May 2018',
       'Jun 2018', 'Jul 2018', 'Aug 2018', 'Sep 2018', 'Oct 2018', 'Nov 2018',
       'Dec 2018', 'Jan 2019'],
      dtype='object')

In [57]:
NEW_NOTES_2.columns

Index(['Core Existing Data', 'Standard_Deviation', 'Feb 2019', 'Apr 2019',
       'May 2019', 'Jun 2019', 'Jul 2019', 'Aug 2019', 'Sep 2019', 'Oct 2019',
       'Nov 2019', 'Dec 2019', 'Jan 2020', 'Feb 2020', 'Mar 2020'],
      dtype='object')

In [58]:
NOTES_2.columns

Index(['Core Existing Data', 'Standard_Deviation', 'Feb 2019', 'Mar 2019',
       'Apr 2019', 'May 2019', 'Jun 2019', 'Jul 2019', 'Aug 2019', 'Sep 2019',
       'Oct 2019', 'Nov 2019', 'Dec 2019', 'Jan 2020', 'Feb 2020'],
      dtype='object')

In [62]:
NOTES_3.columns

Index(['Unnamed: 0', 'Feb 2018', 'Mar 2018', 'Apr 2018', 'May 2018',
       'Jun 2018', 'Jul 2018', 'Aug 2018', 'Sep 2018', 'Oct 2018', 'Nov 2018',
       'Dec 2018', 'Jan 2019', 'Feb 2020'],
      dtype='object')

In [63]:
NOTES_3

,Unnamed: 0,Feb 2018,Mar 2018,Apr 2018,May 2018,Jun 2018,Jul 2018,Aug 2018,Sep 2018,Oct 2018,Nov 2018,Dec 2018,Jan 2019,Feb 2020
0,mailing_completeness,9.807365e-01,9.806970e-01,9.807401e-01,9.807039e-01,9.816793e-01,9.818366e-01,9.819300e-01,9.818695e-01,9.819583e-01,9.819878e-01,9.814942e-01,9.811296e-01,9.813482e-01
1,polo_completeness,9.176149e-01,9.180966e-01,9.182533e-01,9.178518e-01,9.167828e-01,9.157108e-01,9.144094e-01,9.145814e-01,9.149873e-01,9.155264e-01,9.156456e-01,9.139009e-01,8.933039e-01
2,phone_completeness,5.493377e-01,5.540686e-01,5.612384e-01,5.721287e-01,5.878471e-01,6.224581e-01,6.726661e-01,7.101824e-01,7.260111e-01,7.611509e-01,7.612333e-01,7.663104e-01,8.239510e-01
3,fax_completeness,5.217211e-01,5.251197e-01,5.284765e-01,5.354099e-01,5.442739e-01,5.644525e-01,5.899450e-01,6.134177e-01,6.236527e-01,6.499296e-01,6.525742e-01,6.563546e-01,6.843971e-01
4,practice_completeness,9.408156e-01,9.410611e-01,9.413854e-01,9.426681e-01,9.438601e-01,9.434923e-01,9.363291e-01,9.358195e-01,9.357130e-01,9.360464e-01,9.378727e-01,9.381400e-01,9.332388e-01
5,employment_completeness,7.583550e-01,7.596845e-01,7.608581e-01,7.681121e-01,7.713958e-01,7.703520e-01,7.574908e-01,7.555120e-01,7.547828e-01,7.561757e-01,7.585733e-01,7.605247e-01,7.610465e-01
6,spec_completeness,9.687664e-01,9.688731e-01,9.687515e-01,9.525193e-01,9.516404e-01,9.527290e-01,9.581569e-01,9.623254e-01,9.647991e-01,9.652748e-01,9.651287e-01,9.650051e-01,9.630815e-01
7,email_completeness,5.837778e-01,5.904010e-01,5.917554e-01,5.843340e-01,5.872254e-01,5.919233e-01,6.044953e-01,6.093457e-01,6.112017e-01,6.087193e-01,6.352229e-01,6.399271e-01,6.832901e-01
8,polo_correctness,5.520945e-01,5.584764e-01,5.425852e-01,5.874391e-01,6.163484e-01,7.216117e-01,6.906013e-01,6.670481e-01,5.797980e-01,5.502538e-01,8.320675e-01,8.377193e-01,7.884085e-01
9,phone_correctness,6.826029e-01,6.926148e-01,7.220000e-01,7.228514e-01,6.877497e-01,7.776316e-01,7.716895e-01,7.625899e-01,7.473958e-01,6.856420e-01,5.592705e-01,5.377215e-01,5.091278e-01


In [64]:
NOTES_2

,Core Existing Data,Standard_Deviation,Feb 2019,Mar 2019,Apr 2019,May 2019,Jun 2019,Jul 2019,Aug 2019,Sep 2019,Oct 2019,Nov 2019,Dec 2019,Jan 2020,Feb 2020
0,mailing_completeness,0.001016,9.801475e-01,9.799375e-01,9.793197e-01,9.820495e-01,9.823056e-01,9.824251e-01,9.824004e-01,9.823522e-01,9.821872e-01,9.820343e-01,9.820186e-01,9.816848e-01,9.813482e-01
1,polo_completeness,0.006479,9.127862e-01,9.104167e-01,9.093118e-01,9.097503e-01,9.090362e-01,9.074398e-01,9.065507e-01,9.032298e-01,9.014950e-01,8.986148e-01,8.957083e-01,8.938377e-01,8.933039e-01
2,phone_completeness,0.024284,7.695343e-01,7.735810e-01,7.748903e-01,7.662985e-01,7.731660e-01,7.723157e-01,7.847550e-01,7.758147e-01,7.936667e-01,8.230647e-01,8.385015e-01,8.178283e-01,8.239510e-01
3,fax_completeness,0.008914,6.590192e-01,6.626645e-01,6.647006e-01,6.677837e-01,6.712995e-01,6.750114e-01,6.835850e-01,6.804054e-01,6.808750e-01,6.823712e-01,6.844692e-01,6.842981e-01,6.843971e-01
4,practice_compelteness,0.003155,9.368782e-01,9.370330e-01,9.370184e-01,9.382197e-01,9.385640e-01,9.294054e-01,9.286639e-01,9.310614e-01,9.323248e-01,9.327903e-01,9.331049e-01,9.334729e-01,9.332388e-01
5,employment_completeness,0.005368,7.610338e-01,7.636485e-01,7.656256e-01,7.707130e-01,7.718279e-01,7.615926e-01,7.560883e-01,7.524973e-01,7.554930e-01,7.579662e-01,7.601435e-01,7.609208e-01,7.610465e-01
6,spec_completeness,0.007451,9.646244e-01,9.641686e-01,9.635262e-01,9.477665e-01,9.475002e-01,9.477912e-01,9.489178e-01,9.496063e-01,9.629498e-01,9.634377e-01,9.634724e-01,9.633218e-01,9.630815e-01
7,email_completeness,0.018724,6.401563e-01,6.416888e-01,6.424602e-01,6.333600e-01,6.347998e-01,6.365401e-01,6.409586e-01,6.444066e-01,6.658347e-01,6.675823e-01,6.810460e-01,6.824598e-01,6.832901e-01
8,polo_correctness,0.027465,8.322926e-01,8.383562e-01,8.210432e-01,8.370000e-01,8.541667e-01,8.641860e-01,8.428710e-01,8.435973e-01,8.415842e-01,8.392051e-01,9.162349e-01,8.586387e-01,7.884085e-01
9,phone_correctness,0.018839,5.299446e-01,5.387819e-01,5.300050e-01,5.191000e-01,5.362615e-01,5.145186e-01,4.929078e-01,4.928789e-01,5.133929e-01,5.200793e-01,4.735529e-01,5.345000e-01,5.091278e-01
